In [3]:
import datetime
import math
import json
import pptx
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.oxml.xmlchemy import OxmlElement
from pptx.util import Inches, Pt

ModuleNotFoundError: ignored

In [4]:
def load_json(json_file):
    with open(json_file, "r", encoding='utf-8') as read_file:
        return json.load(read_file, strict=False)

In [5]:
def sub_element(parent, tagname, **kwargs):
    element = OxmlElement(tagname)
    element.attrib.update(kwargs)
    parent.append(element)
    return element


In [6]:
def make_para_bullet_pointed(para):
    """Bullets are set to Arial, actual text can be a different font"""
    pPr = para._p.get_or_add_pPr()
    pPr.set('marL', '164000')
    pPr.set('indent', '-122920')
    _ = sub_element(parent=pPr, tagname="a:buFont", typeface="Arial", panose="020B0604020202020204", pitchFamily="34", charset="0" )
    _ = sub_element(parent=pPr, tagname='a:buChar', char="•")

In [7]:
def init_pptx(slide_height, slide_width):
    prs = Presentation()
    prs.slide_height = Inches(slide_height)
    prs.slide_width = Inches(slide_width)
    return prs

In [8]:
def init_slide(prs):
    blank_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_slide_layout)
    slide.shapes.add_picture("sidebar.png", Inches(0), Inches(0))
    slide.shapes.add_picture("talan_logo.png", Inches(11.5), Inches(0.1), width=Inches(1.5), height=Inches(0.7)) # à changer png
    tx_box = slide.shapes.add_textbox(Inches(5.4), Inches(7.2), width=Inches(3), height=Inches(0.25))
    tx_frame = tx_box.text_frame
    tx_frame.word_wrap = True
    paragraph = tx_frame.paragraphs[0]
    date = paragraph.add_run()
    date.text = "Confidentiel, reproduction interdite"
    date.font.bold = False
    date.font.size = Pt(9)
    date.font.color.rgb = RGBColor(0xcd, 0xcd, 0xcd)
    date.font.name = 'Montserrat'
    return slide

In [9]:
def add_title_section(tx_frame_title, txt):
    paragraph_title = tx_frame_title.paragraphs[0]
    title = paragraph_title.add_run()
    title.text += txt.upper()
    title.font.bold = True
    title.font.size = Pt(11)
    title.font.color.rgb = RGBColor(0x48, 0xb2, 0xda)
    title.font.name = 'Montserrat'

In [10]:
def add_title_paragraph(tx_frame_title, txt):
    paragraph_title = tx_frame_title.paragraphs[0]
    title = paragraph_title.add_run()
    title.text = txt
    title.font.bold = True
    title.font.size = Pt(10)
    title.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a)
    title.font.name = 'Montserrat'

In [13]:
def add_bullet_paragraph(tx_frame, consultant):
    paragraph_bullet = tx_frame.add_paragraph()
    bullet = paragraph_bullet.add_run()

    for item in consultant: 
      if consultant.index(item) == len(consultant) - 1: 
        bullet.text += item
      else: 
        bullet.text += item + '\n' 
        bullet.font.bold = False 
        bullet.font.size = Pt(9.5) 
        bullet.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a) 
        bullet.font.name = 'Montserrat' 
        make_para_bullet_pointed(paragraph_bullet)

In [15]:
def make_language(slide, language_dic, x, y, width_language): 
  tx_box = slide.shapes.add_textbox(Inches(x), Inches(y), width=Inches(width_language), height=Inches(0.25)) 
  tx_frame = tx_box.text_frame 
  tx_frame.auto_size = MSO_AUTO_SIZE.SHAPE_TO_FIT_TEXT 
  tx_frame.word_wrap = False
  
  paragraph = tx_frame.paragraphs[0] 
  language = paragraph.add_run() 
  
  language.text = language_dic["nom"] 
  language.font.bold = False 
  language.font.size = Pt(9.5) 
  language.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a) 
  language.font.name = 'Montserrat'

  x_bullet = x + width_language 
  y_bullet = y + 0.096 
  for lvl in range(1, 6): 
    if lvl <= int(language_dic["niveau"]): 
      slide.shapes.add_picture("bullet_on.png", Inches(x_bullet), Inches(y_bullet)) 
    else: 
        slide.shapes.add_picture("bullet_off.png", Inches(x_bullet), Inches(y_bullet)) 
        x_bullet += 0.12
        
  return x_bullet

In [16]:
def align_language(consultant):
    list_width_language = []
    for i in consultant.language:
        if 'i' not in i["nom"]:
            width_language = len(i["nom"]) * 0.105
            list_width_language += [width_language]
        else:
            width_language = len(i["nom"]) * 0.1
            list_width_language += [width_language]
    width_left = []
    width_right = []
    for i in list_width_language:
        if list_width_language.index(i) % 2 == 0:
            width_left += [i]
        else:
            width_right += [i]
    return max(width_right), max(width_left)

In [17]:
def add_language(slide, consultant, x, y):
    p = 1
    language = 0
    if len(consultant.language) !=1:
        width_right = align_language(consultant)[0]
        width_left = align_language(consultant)[1]
        for i in consultant.language:
            if p == 1:
                language = make_language(slide, i, x, y, width_left)
            elif p == 2:
                make_language(slide, i, language, y, width_right)
            else:
                y += 0.2
                make_language(slide, i, x, y, width_left)
            p += 1
    else:
        width_language = len(consultant.language[0]["nom"]) * 0.105 if 'i' not in consultant.language[0]["nom"] else len(consultant.language[0]["nom"]) * 0.1
        make_language(slide, consultant.language[0], x, y, width_language)

In [18]:
def add_education(tx_frame, consultant):
    paragraph_education = tx_frame.add_paragraph()
    eduction_bullet = paragraph_education.add_run()
    for formation in consultant.education:
        if consultant.education.index(formation) == len(consultant.education) - 1:
            eduction_bullet.text += formation["etablissement"] + " - " + formation["diplome"]
        else:
            eduction_bullet.text += formation["etablissement"] + " - " + formation["diplome"] + '\n'
        eduction_bullet.font.bold = False
        eduction_bullet.font.size = Pt(9.5)
        eduction_bullet.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a)
        eduction_bullet.font.name = 'Montserrat'
    make_para_bullet_pointed(paragraph_education)

In [19]:
def add_sectors(tx_frame, consultant):
    paragraph_sectors = tx_frame.add_paragraph()
    sector_bullet = paragraph_sectors.add_run()
    for sector in consultant.sectors:
        if consultant.sectors.index(sector) == len(consultant.sectors) - 1:
            sector_bullet.text += sector
        else:
            sector_bullet.text += sector + ' | '
        sector_bullet.font.bold = False
        sector_bullet.font.size = Pt(9.5)
        sector_bullet.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a)
        sector_bullet.font.name = 'Montserrat'

In [20]:
def make_job_section(slide, job_dic, ante_section_height): 
  section_y = ante_section_height 
  
  tx_box = slide.shapes.add_textbox(Inches(4.5), Inches(section_y), width=Inches(8.5), height=Inches(0.2)) 
  tx_frame = tx_box.text_frame
  tx_frame.word_wrap = True 
  
  paragraph_sectors = tx_frame.paragraphs[0] 
  job_title = paragraph_sectors.add_run() 
  job_title.text = job_dic["title"] + "\n" 
  job_title.font.bold = True 
  job_title.font.size = Pt(9.5) 
  job_title.font.color.rgb = RGBColor(0x29, 0x89, 0xad) 
  job_title.font.name = 'Montserrat' 
  
  add_date_job(slide, job_dic, section_y) 
  
  section_y += 0.20
  tx_box_sub = slide.shapes.add_textbox(Inches(4.5), Inches(section_y), width=Inches(8.5), height=Inches(0.2)) 
  tx_frame_sub = tx_box_sub.text_frame 
  tx_frame_sub.word_wrap = True 
  
  paragraph_objective = tx_frame_sub.paragraphs[0] 
  job_objective = paragraph_objective.add_run() 
  job_objective.text = job_dic["subTitle"] 
  job_objective.font.bold = True 
  job_objective.font.size = Pt(9.5) 
  job_objective.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a) 
  job_objective.font.name = 'Montserrat'
  
  section_y += 0.20 
  return make_job_description(slide, job_dic, section_y)

In [22]:
def make_job_description(slide, job_dic, ante_section_height):
    section_y = ante_section_height
    section_height = 0
    
    for lines in job_dic["description"]:
        section_height += 0.18
        if len(lines) > 115:
            section_height += 0.16*(math.floor(len(lines)/115))
    
    tx_box = slide.shapes.add_textbox(Inches(4.5), Inches(section_y), width=Inches(8.5), height=Inches(section_height))
    tx_frame = tx_box.text_frame
    tx_frame.word_wrap = True
    
    p = 0
    for line in job_dic["description"]:
        if line == "":
            pass
        
        if line[0] == '-':
            if p == 0:
                paragraph_bullet = tx_frame.paragraphs[0]
            else:
                paragraph_bullet = tx_frame.add_paragraph()
            
            bullet = paragraph_bullet.add_run()
            bullet.text = line[2:]
            bullet.font.bold = False
            bullet.font.size = Pt(9.5)
            bullet.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a)
            bullet.font.name = 'Montserrat'
            make_para_bullet_pointed(paragraph_bullet)
        else:
            if p == 0:
                paragraph = tx_frame.paragraphs[0]
            else:
                paragraph = tx_frame.add_paragraph()
            
            description_line = paragraph.add_run()
            description_line.text = line
            description_line.font.bold = False
            description_line.font.size = Pt(9.5)
            description_line.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a)
            description_line.font.name = 'Montserrat'
            
            p += 1
            
    return section_y+section_height

In [23]:
def add_date_job(slide, job_dic, ante_section_height):
    section_y = ante_section_height
    tx_box = slide.shapes.add_textbox(Inches(10.7), Inches(section_y), width=Inches(3), height=Inches(0.25))
    tx_frame = tx_box.text_frame
    tx_frame.word_wrap = True
    
    slide.shapes.add_picture("date_logo.png", Inches(10.5), Inches(section_y), width=Inches(0.2), height=Inches(0.21))
    
    date_cut = job_dic["Date"].split("-")
    start_date = date_cut[0].split("/")
    end_date = date_cut[1].split("/")
    
    start_date = datetime.datetime(int(start_date[1]), int(start_date[0]), 1)
    end_date = datetime.datetime(int(end_date[1]), int(end_date[0]), 1)
    
    total_month_job = ((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month))
    
    if total_month_job >= 12:
        total_year = (total_month_job - total_month_job % 12) / 12
        if total_month_job % 12 == 0 and total_year > 1:
            final_date = str(int(total_year)) + " ans"
        elif total_month_job % 12 == 0 and total_year == 1:
            final_date = str(int(total_year)) + " an"
        elif total_year == 1:
            final_date = str(int(total_year)) + " an et " + str(total_month_job % 12) + " mois"
        else:
            final_date = str(int(total_year)) + " ans et " + str(total_month_job % 12) + " mois"
    else:
        final_date = str(total_month_job) + " mois"
    
    paragraph = tx_frame.paragraphs[0]
    date = paragraph.add_run()
    
    if end_date < start_date:
        date.text = str(date_cut[0].split("/")[0]) + '/' + str(date_cut[0].split("/")[1]) + ' - en cours'
    else:
        date.text = job_dic["Date"] + " - "
    
    date.font.bold = False
    date.font.size = Pt(9.5)
    date.font.color.rgb = RGBColor(0x6a, 0x6a, 0x6a)
    date.font.name = 'Montserrat'
    
    time = paragraph.add_run()
    
    if end_date < start_date:
        time.text = ''
    else:
        time.text = final_date
    
    time.font.bold = False
    time.font.size = Pt(9.5)
    time.font.color.rgb = RGBColor(0x48, 0xb2, 0xda)
    time.font.name = 'Montserrat'